<a href="https://colab.research.google.com/github/SanePai/Neural_Networks-Assignment-2/blob/main/Question_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
%cd "/content/drive/MyDrive/NNFL/Assignment 2"
!ls

/content/drive/.shortcut-targets-by-id/104/Assignment 2
assignment2.pdf  class2_images	class_label.mat  data5.mat     input.mat
class1_images	 class3_images	data55.xlsx	 input_a2.mat  label.mat


In [10]:
import numpy as np
from scipy.io import loadmat
import random
from sklearn.metrics import confusion_matrix as cm

In [11]:
f = loadmat('data5.mat')
D = f['x']
np.random.shuffle(D)

In [12]:
def init_data(data):
    X = np.array(data[:2148, :-1], dtype = float)
    y = np.array(data[:2148, -1], dtype = int)
    X = (X - X.mean(axis = 0))/X.std(axis = 0)
    return X, y

def gaussian(x,center,sigma,beta):
    #Gaussian Kernel
    return np.exp(-beta * (np.linalg.norm(x - center)) ** 2)

def multi_quadric(x, center, sigma, beta):
    #Multi Quadratic Kernel
    return ((np.linalg.norm(x - center)) ** 2 + sigma ** 2) ** 0.5

def linear(x, center, sigma, beta):
    #Linear Kernel
    return np.linalg.norm(x - center)

X_tot, y_tot = init_data(D)

train_X = X_tot[ : 1600]
train_y = y_tot[ : 1600]
test_X = X_tot[1600 : 2148]
test_y = y_tot[1600 : 2148]

def fit_rbf(train_X, train_y, test_X, test_y):
    centers, labels = kmeans(train_X,K=550)

    sigma = np.zeros((len(centers), 1))
    beta = np.zeros((len(centers), 1))
    cluster_size = np.zeros((len(centers), 1))

    for i in range(len(train_X)):
        sigma[labels[i]] += np.linalg.norm(train_X[i] - centers[labels[i]])
        cluster_size[labels[i]] += 1

    sigma /= cluster_size
    beta = 1 / 2 * (sigma * sigma + 1e-6)

    H = np.zeros((len(train_X), len(centers)))

    for i in range(len(train_X)):
        for j in range(len(centers)):
            H[i, j] = linear(train_X[i], centers[j], sigma[j], beta[j])

    W = np.dot(np.linalg.pinv(H), train_y)

    #Test run
    H_test = np.zeros([len(test_X), len(centers)])
    for i in range(len(test_X)):
        for j in range(len(centers)):
            H_test[i, j] = linear(test_X[i], centers[j], sigma[j], beta[j])

    y_pred = np.dot(H_test, W)
    for i in range(len(y_pred)):
        y_pred[i] = 1 if y_pred[i]>=0.5 else 0
        
    accuracy = 0    
    for i in range(len(y_pred)):
        if y_pred[i] == test_y[i]:
            accuracy +=1
    accuracy /= len(y_pred)
    print(accuracy)
    return y_pred, accuracy

In [13]:
def kmeans(X,K=3,max_iter=5000):
    '''
    K= number of clusters
    '''
    m,n = np.shape(X)
    def create_dict(n):
        a = {}
        for i in range(n):
            a[f'{i}'] = []
        return a 
    def calc_distance(A,B):
       return np.linalg.norm(A-B,2)
    
    def return_cluster(C_ind,ind):
        '''
        Returns the cluster that the index belongs to
        '''
        for i in C_ind.keys():
            if ind in C_ind[i]:
                return int(i)


    #random centers
    index_random = np.random.randint(0,m,K)
    C = []
    #Initializing random cluster centers
    for i in range(K):
        C.append(X[index_random[i]])
    #New Centers
    NC = np.zeros((K,n))

    #converged state bool
    st = False
    iter = 0
    while True:
        if iter == max_iter or st:
            break
            
        C_ind = create_dict(K)
        for i in range(m):
            d = []
            for j in range(K):
                d.append(calc_distance(X[i],C[j]))
            index = np.argmin(d)
            C_ind[f'{index}'].append(i) 

        for i in range(K):
            c1_ind = C_ind[f'{i}']
            if len(c1_ind):
                for j in range(len(c1_ind)):
                    NC[i] += X[c1_ind[j]]
                NC[i] /= len(c1_ind)

        if (C==NC).all():
            st = True
        
        C = NC
        iter += 1
    
    # return np.array(C),(C_ind)
    labels = []
    for i in range(m):
        labels.append(return_cluster(C_ind,i))
    return C,labels

In [15]:
y_pred, _ = fit_rbf(train_X, train_y, test_X, test_y)
for i in range(len(y_pred)):
    y_pred[i] = 1 if y_pred[i] > 0.5 else 0

confmat = cm(test_y, y_pred)

Accuracy = (confmat[0][0]+confmat[1][1])/len(y_pred)
Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
print("Confusion Matrix:")
print(confmat)
print("\n")
print(f"Accuracy: {Accuracy}\nSensitivity: {Sensitivity}\nSpecificity: {Specificity}\n")

avg_acc = 0
# K - fold cross validation

for k in range(5):
    X = X_tot[0 : 1718]
    y = y_tot[0 : 1718]
    X_val = X_tot[1718 :]
    y_val = y_tot[1718 :]
    print(f'Fold {k+1}:')
    print("Accuracy: ", end="")
    _, acc = fit_rbf(X, y, X_val, y_val)
    print("===================")
    avg_acc += acc
    X_tot[0 : 430] = X_val
    X_tot[430 : ] = X
    y_tot[0 : 430] = y_val
    y_tot[430 : ] = y

avg_acc /= 5
print(f'\nAverage Accuracy: {avg_acc*100}%')

0.9470802919708029
Confusion Matrix:
[[264  17]
 [ 12 255]]


Accuracy: 0.9470802919708029
Sensitivity: 0.9550561797752809
Specificity: 0.9395017793594306

Fold 1:
Accuracy: 0.958139534883721
Fold 2:
Accuracy: 0.9232558139534883
Fold 3:
Accuracy: 0.9651162790697675
Fold 4:
Accuracy: 0.9720930232558139
Fold 5:
Accuracy: 0.9511627906976744

Average Accuracy: 95.3953488372093%
